In [1]:
import re
import os
from multiprocessing import Pool
from glom import glom

import numpy as np
import pandas as pd


import yaml
import json
import xmltodict

import requests
from urllib.request import urlretrieve, urlopen

In [2]:
folder = '../data/'
def forceDir(dirName):
    if not os.path.exists(dirName):
        os.makedirs(dirName)
    return dirName


index_folder = forceDir('us990/index/')
bmf_folder = forceDir('bmf/')
whitelist_folder = forceDir('whitelist/')
lists_folder = forceDir('lists/')
f990_folder = forceDir('../data/us990/')
json_folder = forceDir('us990/json/')


In [3]:
bmf = pd.read_pickle(bmf_folder + 'missouri_bmf.pkl')
print(bmf.shape)
bmf.head(3)

(16122, 27)


,NAME,ICO,STREET,CITY,STATE,ZIP,GROUP,SUBSECTION,AFFILIATION,CLASSIFICATION,...,ASSET_CD,INCOME_CD,FILING_REQ_CD,PF_FILING_REQ_CD,ACCT_PD,ASSET_AMT,INCOME_AMT,REVENUE_AMT,NTEE_CD,SORT_NAME
EIN,,,,,,,,,,,,,,,,,,,,,
000360268,IGLESIA VICTORIA,NaN,1445 N BOONVILLE AVE,SPRINGFIELD,MO,65802-1894,1678,3,9,7000,...,0,0,6,0,12,NaN,NaN,NaN,NaN,NaN
010605038,HABITAT FOR HUMANITY INTERNATIONAL INC,% EXECUTIVE DIRECTOR,PO BOX 223,CASSVILLE,MO,65625-0223,8545,3,9,1000,...,4,4,1,0,6,413721.0,116609.0,116609.0,L20Z,HFH OF SOUTH BARRY CO MO
010829991,MORLEY CHURCH OF GOD,% JEANNIE E WILLIAMS,PO BOX 267,MORLEY,MO,63767-0267,1489,3,9,7000,...,0,0,6,0,12,NaN,NaN,NaN,NaN,NaN


In [4]:
annual_index = 'https://s3.amazonaws.com/irs-form-990/index_%d.json'

dfs = []
for year in range(2014, 2020):
    response = urlopen(annual_index % year)
    data = json.load(response)
    __year, data = data.popitem()
    df = pd.DataFrame(data)
    df['source'] = year
    dfs.append(df)
df = pd.concat(dfs, ignore_index=True)
print(df.shape)
df.head(2)

(2233442, 10)


,EIN,TaxPeriod,DLN,FormType,URL,OrganizationName,SubmittedOn,ObjectId,LastUpdated,source
0,760740805,201406,93492311012114,990EZ,https://s3.amazonaws.com/irs-form-990/20141311...,ROYAL HIGH BAND BOOSTERS,2014-12-08,201413119349201211,2016-03-21T17:23:53,2014
1,996001533,201406,93492311012164,990EZ,https://s3.amazonaws.com/irs-form-990/20141311...,GARDEN CLUB OF HONOLULU,2014-12-08,201413119349201216,2016-03-21T17:23:53,2014


In [5]:
df.EIN = df.EIN.astype(str).str.pad(9, fillchar='0')
df = df.set_index('EIN')

In [6]:
df.to_pickle(f990_folder + 'index.pkl')
df.to_csv(f990_folder + 'index.csv')

### Intersect MO-bmf with available forms

In [9]:
good_ein = bmf.index.tolist()

In [12]:
intersected_index = df[df.index.isin(good_ein)]
intersected_index.to_pickle(f990_folder + 'intersected_index.pkl')
intersected_index.to_csv(f990_folder + 'intersected_index.csv')
print(intersected_index.shape)
intersected_index.head()

(16250, 9)


,TaxPeriod,DLN,FormType,URL,OrganizationName,SubmittedOn,ObjectId,LastUpdated,source
EIN,,,,,,,,,
431414775,201312,93493314020054,990,https://s3.amazonaws.com/irs-form-990/20140314...,ST LOUIS COUNTY FIRE ACADEMY INC,2014-12-06,201403149349302005,2016-03-21T17:23:53,2014
431664669,201312,93493314029124,990,https://s3.amazonaws.com/irs-form-990/20142314...,BOYS AND GIRLS CLUB OF THE OZARKS,2014-12-06,201423149349302912,2016-03-21T17:23:53,2014
371538010,201406,93492317026184,990EZ,https://s3.amazonaws.com/irs-form-990/20143317...,FRIENDSHIP HALL INC,2014-12-08,201433179349202618,2016-03-21T17:23:53,2014
431200992,201406,93492315010334,990EZ,https://s3.amazonaws.com/irs-form-990/20143315...,CHEROKEE PASS FIRE DISTRICT,2014-12-08,201433159349201033,2016-03-21T17:23:53,2014
237213595,201406,93492314010914,990EZ,https://s3.amazonaws.com/irs-form-990/20141314...,KIRKSVILLE ROTARY CLUB,2014-12-08,201413149349201091,2016-03-21T17:23:53,2014
